In [1]:
cd ../..

/people/gerald/Documents/repositories/Educational-French-Question-Answering


In [2]:
import os
import json
import evaluate
import random
from src.data_utils.pb_corpus import FQAGPBDataset
from src.data_utils.corpus import MixedDataset, KeyMapDataset
from src.model.mbart_qg import MBARTQGDataLoaderCollator

In [3]:
os.environ['EFQADATA'] = '/people/gerald/Documents/repositories/Educational-French-Question-Answering/dataset'
os.environ['QA_LOG'] = "/data/workdir/gerald/log"

In [4]:

data_folder = os.path.expandvars("$EFQADATA/source")
datasets_name = ["squad-en-en.pb.json","fquad-fr-fr.pb.json"]
datasets = {}

split = "train"

for dataset_name in datasets_name: 
    with open(os.path.join(data_folder, dataset_name)) as f:
        data = json.load(f)
        datasets[dataset_name.split('.')[0]] = FQAGPBDataset(data['train'], sampler = lambda x : [x[random.randint(0, len(x) - 1)]])


In [5]:
md = KeyMapDataset(MixedDataset(*datasets.values()))

In [6]:
md[0]

{'is_default': True,
 'id': 0,
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to <hl>Saint Bernadette Soubirous<hl> in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question_type': 'NONE',
 'input_lang': 250008,
 'output_lang': 250008,
 'index': 0,
 'dataset_index': 0}

In [7]:
from torch.utils.data import DataLoader
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [8]:
dl  = DataLoader(md, batch_size = 2, shuffle=True, collate_fn=MBARTQGDataLoaderCollator(tokenizer))

In [9]:
next(iter(dl))

{'input_ids': tensor([[250008,   9020,   1556, 112730,     10,  88551,  31486,     23,     70,
            1346,  53099,      4,    136,   1556,  13036,  12275,      7,     99,
           54511,   2069,    136,     10,   5361,  93425,    136,   1305,      9,
           57877,   1830,  26908,  11698,     71,     23,   1061,    497,      5,
           27985,    214,  48962,   3847,      7,   1556,   6863,  10336, 167565,
            1314,     23,   9020,      5,   9020,   1556,   2809,     70,  53550,
             100,  54180,  26719,  70189,    384,  16196,     18,   5173,  72208,
               4,  38213,  16635,    429,   5173,  62634,      4,   7948,   4378,
            5173,  71204,      4,    581,  21027, 100974,  55283,      7,     15,
          160733,    247,   2646,  34997,  40073,   5173,  88235,      4,  23213,
           11584,   5128,      7,   5173,  88235,      4,    335,  17336,   2037,
            5173,  84308,      4,    581,  14407,  18621,  41626, 223265,      4,
   

In [10]:
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

from src.model.mbart_qg import MBARTQG, MBARTQGDataLoaderCollator
from src.eval_utils.evaluate_utils import HFMetric, MultiHFMetric

import spacy
os.environ["TOKENIZERS_PARALLELISM"] = "false"
class SpacyTokenizer():
    def __init__(self):
        self.nlp = spacy.load("fr_core_news_lg")
    def __call__(self, x):
        return [t.text for t in self.nlp.tokenizer(x)]
st = SpacyTokenizer()
validation_metrics = MultiHFMetric(
    sacrebleu = HFMetric('sacrebleu', lambda x : x['score'], tokenize = 'intl'),
    rouge = HFMetric('rouge', lambda x : x['rougeL'], tokenizer = st)
)

os.environ['EFQADATA'] = '/people/gerald/Documents/repositories/Educational-French-Question-Answering/dataset'
data_folder = os.path.expandvars("$EFQADATA/source")
train_datasets_name = ["squad-en-en.pb.json","fquad-fr-fr.pb.json"]
valid_datasets_name = ["fquad-fr-fr.pb.json"]
train_datasets = {}
valid_datasets = {}

for dataset_name in train_datasets_name: 
    with open(os.path.join(data_folder, dataset_name)) as f:
        il, ol = dataset_name.split('.')[0].split('-')[-2], dataset_name.split('.')[0].split('-')[-1]
        data = json.load(f)
        train_datasets[dataset_name.split('.')[0]] = FQAGPBDataset(
            data["train"],
            sampler = lambda x : [x[random.randint(0, len(x) - 1)]],
            input_lang = il, output_lang = ol
        )
for dataset_name in valid_datasets_name: 
    with open(os.path.join(data_folder, dataset_name)) as f:
        il, ol = dataset_name.split('.')[0].split('-')[-2], dataset_name.split('.')[0].split('-')[-1]
        data = json.load(f)
        valid_datasets[dataset_name.split('.')[0]] = FQAGPBDataset(
            data["valid"],
            sampler = lambda x : [x[random.randint(0, len(x) - 1)]],
            input_lang = il, output_lang = ol
        )

model = MBARTQG(
    pretrained_name = "facebook/mbart-large-50-many-to-many-mmt",
    fixed_encoder = True,
    validation_callback = validation_metrics, log_dir = os.path.join(os.path.expandvars("$QA_LOG"), 'test')
)

tb_logger = pl_loggers.TensorBoardLogger(save_dir=os.path.expandvars("$QA_LOG"), name="test")
tb_logger.log_hyperparams({"test": 1 })
lr_monitor = LearningRateMonitor(logging_interval='step')

train_dl  = DataLoader(KeyMapDataset(MixedDataset(*train_datasets.values())), batch_size = 2, shuffle=True, num_workers=8, collate_fn=MBARTQGDataLoaderCollator(model.tokenizer))
valid_dl  = DataLoader(KeyMapDataset(MixedDataset(*valid_datasets.values())), batch_size = 2, shuffle=False, num_workers=8, collate_fn=MBARTQGDataLoaderCollator(model.tokenizer))


checkpoint_callback_val_loss = ModelCheckpoint(monitor='val/loss', save_top_k=2, mode="min", filename="val-loss-checkpoint-{epoch:02d}-{val_loss:.2f}")
checkpoint_callback_val_sacrebleu = ModelCheckpoint(monitor='val/sacrebleu', save_top_k=2, mode="max", filename="val-sacrebleu-checkpoint-{epoch:02d}-{val_loss:.2f}")
checkpoint_callback_val_rouge = ModelCheckpoint(monitor='val/rouge', save_top_k=2, mode="max", filename="val-rouge-checkpoint-{epoch:02d}-{val_loss:.2f}")

callbacks = [
    lr_monitor,
    checkpoint_callback_val_loss,
    checkpoint_callback_val_rouge,
    checkpoint_callback_val_sacrebleu
]



In [ ]:
trainer = pl.Trainer(
    logger=tb_logger, 
    log_every_n_steps=1, 
    callbacks=callbacks, 
    enable_progress_bar=True,
    limit_train_batches=10000, 
    max_epochs=250, 
    accumulate_grad_batches=64,
    accelerator='gpu',
    devices=[1]
)
trainer.fit(
    model,
    train_dl,
    valid_dl
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/people/gerald/lib/miniconda3/envs/annotationbob/lib/python3.8/site-packages/torch/optim/sgd.py:105: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(SGD, self).__init__(params, defaults)

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | MBartForConditionalGeneration | 610 M 
--------------------------------------------------------
610 M     Trainable params
0         Non-trainable params
610 M     Total params
2,443.522 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/people/gerald/lib/miniconda3/envs/annotationbob/lib/python3.8/site-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Training: 0it [00:00, ?it/s]

In [ ]:
train